# 库导入

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.optimize as sco
from joblib import Parallel, delayed
import multiprocessing
import datetime
import statsmodels.api as sm
import seaborn
from sklearn.metrics import classification_report
import talib as ta
from collections import OrderedDict #保持Key的顺序
import time
# from WindPy import *
# w.start()

from scipy import stats

plt.rcParams['font.sans-serif']=['SimHei']
plt.rcParams['axes.unicode_minus'] = False

# pd.set_option('display.height',1000)
pd.set_option('display.max_rows',500)
pd.set_option('display.max_columns',500)
pd.set_option('display.width',1000)

# 相关交易日

In [41]:
TradeDateDf = pd.read_csv('D:/DWDB/%s.csv'%('AShareCalendar'))
TradeDateDf = TradeDateDf[TradeDateDf['S_INFO_EXCHMARKET'] == 'SSE']
TradeDateDf_D = TradeDateDf['TRADE_DAYS'].apply(lambda x:datetime.datetime.strptime(str(x),'%Y%m%d')).to_frame('TRADE_DAYS').sort_values('TRADE_DAYS')

TradeDateDf_D_Used = TradeDateDf_D[(TradeDateDf_D['TRADE_DAYS']>='2005-01-01') & (TradeDateDf_D['TRADE_DAYS']<='2019-07-31')]
TradeDateDf_D_Used.index = TradeDateDf_D_Used['TRADE_DAYS']
TradeDateDf_D_Used.index.names = ['NOT_TRADE_DAYS']

TradeDateDf_M_Used = TradeDateDf_D_Used.resample('M').last()
TradeDateDf_M_Used.index.names = ['NOT_TRADE_DAYS']

# 日期转换函数
该函数将普通日期转化为下一个月末交易日，便于关联到其他表中

In [3]:
# 月度非标准数据的日期转变为当月（若t>t_end，下个月）最后一个交易日
def MdateToMENDdate(FinancialIndicatorDf,TradeDateDf_M_Used,colname):
    '''
    第1参数，DF，包含 colname 列，公告日期，
    第2参数，DF，包含 TRADE_DAYS 列，为月度交易日，
    '''
    FinancialIndicatorDf_1 = FinancialIndicatorDf.copy()
    FinancialIndicatorDf_end = FinancialIndicatorDf.copy()
    ## 可更改，根据ANN_DT列是否为日期
    try:
        FinancialIndicatorDf_1['TRADE_DAYS_tmp'] = FinancialIndicatorDf_1[colname].apply(lambda x:datetime.datetime.strptime(str(x),'%Y%m%d'))
    except:
        FinancialIndicatorDf_1['TRADE_DAYS_tmp'] = FinancialIndicatorDf_1[colname]
    #################################
    TradeDateDf_M_Used_ = TradeDateDf_M_Used.copy()
    TradeDateDf_M_Used_['TRADE_DAYS_Y_M'] = [each.strftime('%Y-%m-%d')[:7] for each in TradeDateDf_M_Used['TRADE_DAYS'].tolist()].copy()
    FinancialIndicatorDf_1['TRADE_DAYS_Y_M'] = [each.strftime('%Y-%m-%d')[:7] for each in FinancialIndicatorDf_1['TRADE_DAYS_tmp'].tolist()]
    FinancialIndicatorDf_2 = FinancialIndicatorDf_1.merge(TradeDateDf_M_Used_,on=['TRADE_DAYS_Y_M'],how='left')
    TradeDateDf_M_Used_DateList = TradeDateDf_M_Used_['TRADE_DAYS'].tolist()
    FinancialIndicatorDf_3 = FinancialIndicatorDf_2.copy()
#     FinancialIndicatorDf_3['TRADE_DAYS'] = FinancialIndicatorDf_3.apply(lambda x: 
#                                                                         TradeDateDf_M_Used_DateList[TradeDateDf_M_Used_DateList.index(x['TRADE_DAYS'])+1] 
#                                                                         if x['TRADE_DAYS_tmp']>x['TRADE_DAYS'] 
#                                                                         else x['TRADE_DAYS'] ,axis=1)
    FinancialIndicatorDf_end[colname]= FinancialIndicatorDf_3.apply(lambda x: 
                                                                        TradeDateDf_M_Used_DateList[TradeDateDf_M_Used_DateList.index(x['TRADE_DAYS'])+1] 
                                                                        if x['TRADE_DAYS_tmp']>x['TRADE_DAYS'] 
                                                                        else x['TRADE_DAYS'] ,axis=1)
    return FinancialIndicatorDf_end

# 设定因子计算股票池-全A股
1 剔除上市未满6个月  
2 有特殊处理股票  
3 停牌股票  

In [4]:
# 以行情信息对应的日度股票列表筛选得到月度可选股列表
DfForStock = pd.read_csv('D:/DWDB/%s.csv'%('AShareEODPrices'))
DfForStock_0 = DfForStock[['S_INFO_WINDCODE','TRADE_DT']].copy()
DfForStock_0['TRADE_DAYS'] = DfForStock_0['TRADE_DT'].apply(lambda x:datetime.datetime.strptime(str(x),'%Y%m%d'))
DfForStock_0=DfForStock_0[['S_INFO_WINDCODE','TRADE_DAYS']].copy()

In [5]:
# 日度股票列表月度化
DfForStock_1=DfForStock_0.set_index('TRADE_DAYS')
DfForStock_1['TRADE_DAYS']=DfForStock_1.index
DfForStock_2=DfForStock_1.groupby('S_INFO_WINDCODE').apply(lambda x:x.resample('M').last())
DfForStock_3=DfForStock_2.reset_index(drop=True)

## 剔除上市未满6个月的股票

In [6]:
IPODF = pd.read_csv('D:/DWDB/%s.csv'%'AShareIPO')
IPODF['S_IPO_LISTDATE']=IPODF['S_IPO_LISTDATE'].apply(lambda x:datetime.datetime.strptime(str(int(x)),'%Y%m%d') if not np.isnan(x) else np.nan)
IPODF_0 = IPODF[['S_INFO_WINDCODE','S_IPO_LISTDATE']]

DfForStock_4 = DfForStock_3.merge(IPODF_0,on=['S_INFO_WINDCODE'],how='left')
DfForStock_4['IPOtitlNow'] = DfForStock_4['TRADE_DAYS']-DfForStock_4['S_IPO_LISTDATE']
DfForStock_4['IPOtitlNowIF'] = DfForStock_4['IPOtitlNow'].apply(lambda x: 1 if x.days>182 else 0)

DfForStock_5 = DfForStock_4[DfForStock_4['IPOtitlNowIF']==1][['S_INFO_WINDCODE','TRADE_DAYS']].copy()

## 剔除有特殊处理的股票

In [178]:
STDF = pd.read_csv('D:/DWDB/%s.csv'%'AShareST')
STDF['ENTRY_DT'] = STDF['ENTRY_DT'].apply(lambda x:datetime.datetime.strptime(str(int(x)),'%Y%m%d') if not np.isnan(x) else np.nan)
STDF['REMOVE_DT'] = STDF['REMOVE_DT'].apply(lambda x:datetime.datetime.strptime(str(int(x)),'%Y%m%d') if not np.isnan(x) else np.nan)
STDF_0 = STDF[['S_INFO_WINDCODE','ENTRY_DT','REMOVE_DT','S_TYPE_ST']].copy()
STDF_0['REMOVE_DT'] = STDF_0['REMOVE_DT'].fillna(datetime.datetime.strptime('2099-12-31','%Y-%m-%d'))

In [179]:
DfForStock_6=DfForStock_5.merge(STDF_0,left_on=['TRADE_DAYS','S_INFO_WINDCODE'],right_on=['ENTRY_DT','S_INFO_WINDCODE'],how='left')
DfForStock_6=DfForStock_6[DfForStock_6['ENTRY_DT'].isnull()][['TRADE_DAYS','S_INFO_WINDCODE']].copy()
DfForStock_7=DfForStock_6.merge(MdateToMENDdate(STDF_0,TradeDateDf_M_Used,'ENTRY_DT'),left_on=['TRADE_DAYS','S_INFO_WINDCODE'],right_on=['ENTRY_DT','S_INFO_WINDCODE'],how='left')

In [180]:
def FillNaT(df):
    df_=df.sort_values('TRADE_DAYS').copy()
    df_=df_.fillna(method='ffill')
    return df_
DfForStock_8=DfForStock_7.groupby('S_INFO_WINDCODE').apply(FillNaT).reset_index(drop=True)

In [181]:
DfForStock_9=DfForStock_8[~(DfForStock_8['TRADE_DAYS']<=DfForStock_8['REMOVE_DT'])][['S_INFO_WINDCODE','TRADE_DAYS']].copy()

## 剔除停牌的股票

In [183]:
TradingSuspensionDF = pd.read_csv('D:/DWDB/%s.csv'%'AShareTradingSuspension')
TradingSuspensionDF_0 = TradingSuspensionDF[['S_INFO_WINDCODE','S_DQ_SUSPENDDATE','S_DQ_RESUMPDATE']].copy()
TradingSuspensionDF_0['S_DQ_SUSPENDDATE']=TradingSuspensionDF_0['S_DQ_SUSPENDDATE'].apply(lambda x:datetime.datetime.strptime(str(int(x)),'%Y%m%d') if not np.isnan(x) else np.nan)
TradingSuspensionDF_0['S_DQ_RESUMPDATE']=TradingSuspensionDF_0['S_DQ_RESUMPDATE'].apply(lambda x:datetime.datetime.strptime(str(int(x)),'%Y%m%d') if not np.isnan(x) else np.nan)

In [184]:
TradingSuspensionDF_1=TradingSuspensionDF_0.copy()
TradingSuspensionDF_1['S_DQ_RESUMPDATE']=TradingSuspensionDF_1.apply(lambda x: x['S_DQ_SUSPENDDATE'] if pd.isnull(x['S_DQ_RESUMPDATE']) else x['S_DQ_RESUMPDATE'],axis=1)

In [185]:
TradingSuspensionDF_2=MdateToMENDdate(TradingSuspensionDF_1,TradeDateDf_M_Used,'S_DQ_SUSPENDDATE')
TradingSuspensionDF_3=TradingSuspensionDF_2[~(TradingSuspensionDF_2['S_DQ_SUSPENDDATE']>TradingSuspensionDF_2['S_DQ_RESUMPDATE'])]

In [186]:
DfForStock_10=DfForStock_9.merge(TradingSuspensionDF_3,left_on=['TRADE_DAYS','S_INFO_WINDCODE'],right_on=['S_DQ_SUSPENDDATE','S_INFO_WINDCODE'],how='left')
DfForStock_11=DfForStock_10.groupby('S_INFO_WINDCODE').apply(FillNaT).reset_index(drop=True)
DfForStock_12=DfForStock_11[~(DfForStock_11['TRADE_DAYS']<=DfForStock_11['S_DQ_RESUMPDATE'])][['S_INFO_WINDCODE','TRADE_DAYS']].copy()

In [187]:
DfForStock_V = DfForStock_12.copy()

# 因子计算

## 市值因子
$ SIZE=ln(cap) $

In [16]:
# 因子数据获取
DerivativeIndicatorDf = pd.read_csv('D:/DWDB/%s.csv'%('AShareEODDerivativeIndicator'))

In [17]:
# 仅保留所需因子数据
DerivativeIndicator_UsedList = ['S_VAL_MV','S_VAL_PE_TTM','S_VAL_PS_TTM','S_VAL_PB_NEW','S_DQ_FREETURNOVER',]
DerivativeIndicatorDf_0 = DerivativeIndicatorDf[['S_INFO_WINDCODE','TRADE_DT']+DerivativeIndicator_UsedList].copy()
DerivativeIndicatorDf_0['TRADE_DAYS'] = DerivativeIndicatorDf_0['TRADE_DT'].apply(lambda x:datetime.datetime.strptime(str(x),'%Y%m%d'))

In [189]:
BarraFactor_0=DfForStock_V.merge(DerivativeIndicatorDf_0[['S_INFO_WINDCODE','TRADE_DAYS','S_VAL_MV']],on=['S_INFO_WINDCODE','TRADE_DAYS'],how='left')
BarraFactor_0['SIZE']=np.log(BarraFactor_0['S_VAL_MV'])
BarraFactor_0=BarraFactor_0[['S_INFO_WINDCODE','TRADE_DAYS','SIZE']]

In [190]:
BarraFactor_0.to_csv('D:/StockDataBase/BarraFactor_0.csv',index=False)

## 贝塔因子
$$ \beta_t=\frac{\sum_{i=0}^{N-1}\omega_i(x_{t-i}-\bar{x})(y_{t-i}-\bar{y})}{\sum_{i=0}^{N-1}\omega_i(x_{t-i}-\bar{x})^2} $$
$$ \omega_i 为指数加权的权重，此处N=252，半衰期63 $$

In [191]:
# 指数行情数据，市场收益率
IndexEODPrices = pd.read_csv('D:/DWDB/%s.csv'%('AIndexEODPrices'))
Index_0=IndexEODPrices[IndexEODPrices['S_INFO_WINDCODE']=='000002.SH'][['S_INFO_WINDCODE','TRADE_DT','S_DQ_CLOSE']].sort_values('TRADE_DT').copy()
Index_0['TRADE_DAYS']=Index_0['TRADE_DT'].apply(lambda x:datetime.datetime.strptime(str(int(x)),'%Y%m%d') if not np.isnan(x) else np.nan)
Index_0['RET']=Index_0.sort_values('TRADE_DAYS')['S_DQ_CLOSE'].pct_change()
Index_1=Index_0.dropna()[['TRADE_DAYS','RET']].copy().rename(columns={'RET':'I_RET',})

In [192]:
# 个股行情数据，个股收益率
DfForRet = DfForStock[['S_INFO_WINDCODE','TRADE_DT','S_DQ_ADJCLOSE']].copy()
DfForRet['TRADE_DAYS']=DfForRet['TRADE_DT'].apply(lambda x:datetime.datetime.strptime(str(int(x)),'%Y%m%d') if not np.isnan(x) else np.nan)
def ComputeRet(df,datename,colname):
    df_=df.sort_values(datename).copy()
    df_['Ret']=df_[colname]/df_[colname].shift(1)-1
    return df_
DfForRet_0=DfForRet.groupby('S_INFO_WINDCODE').apply(ComputeRet,datename='TRADE_DAYS',colname='S_DQ_ADJCLOSE').reset_index(drop=True)

In [193]:
DfForRet_1=DfForRet_0.dropna()

In [194]:
DfForRet_2=DfForRet_1.merge(Index_1,on=['TRADE_DAYS'],how='left')

In [195]:
DfForRet_2.to_csv('D:/StockDataBase/DfForRet_2.csv',index=False)

In [198]:
def ComputeBeta(df):
    df_=df.sort_values('TRADE_DAYS').copy()
    x=df_['Ret']
    y=df_['I_RET']
    x_bar=x.mean()
    y_bar=y.mean()
    x_=x-x_bar
    y_=y-y_bar
    x_y=(x_*y_).to_frame('x_y').ewm(span=63,adjust=True,min_periods=252).mean()['x_y'].tolist()[-1]
    x_x=(x_*x_).to_frame('x_x').ewm(span=63,adjust=True,min_periods=252).mean()['x_x'].tolist()[-1]
    if x_x==0:
        BETA=np.nan
    else:
        BETA=x_y/x_x
    return pd.DataFrame([df_['S_INFO_WINDCODE'].tolist()[0],BETA],index=['S_INFO_WINDCODE','BETA']).T
def applyParallel(dfGrouped, func):
    retLst = Parallel(n_jobs=multiprocessing.cpu_count()-2)(delayed(func)(group) for name, group in dfGrouped)
    return pd.concat(retLst,axis=0)
def ComputeMfWithDData(TradeDateDf_M_Used,DfForRet_2,ComputeBeta,StartN):
    '''
    日度数据计算月度因子专用函数
    TradeDateDf_M_Used,循环交易日DF
    DfForRet_2,原材料数据集
    StartN,因子计算所需窗口长度
    需要修改groupby后的apply函数
    '''
    DateList=TradeDateDf_M_Used['TRADE_DAYS']
    SaveDf=pd.DataFrame()
    for DateCnt in range(StartN,len(DateList)):
#     for DateCnt in range(StartN,15):
        DateNow=DateList[DateCnt]
        DateLast=DateList[DateCnt-13]
        DfForRet_2_sub=DfForRet_2[(DfForRet_2['TRADE_DAYS']<=DateNow)&(DfForRet_2['TRADE_DAYS']>=DateLast)].copy()
        DfForRet_2_sub0=DfForRet_2_sub[DfForRet_2_sub['S_INFO_WINDCODE'].isin(DfForStock_V[DfForStock_V['TRADE_DAYS']==DateNow]['S_INFO_WINDCODE'])]
        DfForRet_2_sub1=applyParallel(DfForRet_2_sub0.groupby('S_INFO_WINDCODE'),ComputeBeta)
        DfForRet_2_sub1['TRADE_DAYS']=DateNow
        SaveDf=pd.concat([SaveDf,DfForRet_2_sub1],axis=0)
    return SaveDf

In [199]:
DfForRet_3=ComputeMfWithDData(TradeDateDf_M_Used,DfForRet_2,ComputeBeta,StartN=13)

In [200]:
BarraFactor_1=BarraFactor_0.merge(DfForRet_3,on=['S_INFO_WINDCODE','TRADE_DAYS'],how='left')

## 动量因子
$$ MOM=\sum_{i=21}^{252}\omega_iln(1+r_{t-i}) $$

In [201]:
def ComputeMom(df):
    df_=df.sort_values('TRADE_DAYS').copy()
    try:
        MOM=df_['Ret'][-252:-20].apply(lambda x: np.log(1+x)).ewm(span=63,adjust=False,min_periods=225).mean().tolist()[-1]
    except:
        MOM=np.nan
    return pd.DataFrame([df_['S_INFO_WINDCODE'].tolist()[0],MOM],index=['S_INFO_WINDCODE','MOM']).T
DfForRet_4=ComputeMfWithDData(TradeDateDf_M_Used,DfForRet_0,ComputeMom,StartN=13)

In [202]:
BarraFactor_2=BarraFactor_1.merge(DfForRet_4,on=['S_INFO_WINDCODE','TRADE_DAYS'],how='left')

## 残差波动因子
$$ ResVOL=\frac{2}{3}dastd+\frac{1}{6}crma+\frac{1}{6}hsigma $$

In [203]:
def ComputeDastd(df):
    df_=df.sort_values('TRADE_DAYS').copy()
    try:
        Dastd=df_['Ret'][-252:].ewm(span=42,adjust=False,min_periods=225).std().tolist()[-1]
#         Dastd=(((df_['Ret'][-252:]-df_['Ret'][-252:].mean())**2).ewm(span=42,adjust=False,min_periods=225).mean().tolist()[-1])**0.5
    except:
        Dastd=np.nan
    return pd.DataFrame([df_['S_INFO_WINDCODE'].tolist()[0],Dastd],index=['S_INFO_WINDCODE','Dastd']).T
DfForRet_5=ComputeMfWithDData(TradeDateDf_M_Used,DfForRet_0,ComputeDastd,StartN=13)

In [204]:
def ComputeCrma(df):
    df_=df.sort_values('TRADE_DAYS').copy()
    try:
        Zt_series=(df_['Ret'][-252:]+1).apply(lambda x: np.log(1+x)).cumsum()
        Crma=np.log(1+Zt_series.max())-np.log(1+Zt_series.min())
    except:
        Crma=np.nan
    return pd.DataFrame([df_['S_INFO_WINDCODE'].tolist()[0],Crma],index=['S_INFO_WINDCODE','Crma']).T
DfForRet_6=ComputeMfWithDData(TradeDateDf_M_Used,DfForRet_0,ComputeCrma,StartN=13)

In [205]:
def ComputeHsigma(df):
    df_=df.sort_values('TRADE_DAYS').copy()
    try:
        y=df_['I_RET'][-252:]
        x=df_['Ret'][-252:]
        x_c=sm.add_constant(x)
        Hsigma=sm.OLS(y,x_c).fit().resid.std()
    except:
        Hsigma=np.nan
    return pd.DataFrame([df_['S_INFO_WINDCODE'].tolist()[0],Hsigma],index=['S_INFO_WINDCODE','Hsigma']).T
DfForRet_7=ComputeMfWithDData(TradeDateDf_M_Used,DfForRet_2,ComputeHsigma,StartN=13)

In [206]:
DfForRet_8=DfForRet_5.merge(DfForRet_6,on=['S_INFO_WINDCODE','TRADE_DAYS'],how='outer').merge(DfForRet_7,on=['S_INFO_WINDCODE','TRADE_DAYS'],how='outer')
DfForRet_8['ResVOL']=DfForRet_8['Dastd']/3+DfForRet_8['Crma']/6+DfForRet_8['Hsigma']/6

In [207]:
BarraFactor_3=BarraFactor_2.merge(DfForRet_8[['S_INFO_WINDCODE','TRADE_DAYS','ResVOL']],on=['S_INFO_WINDCODE','TRADE_DAYS'],how='left')

In [208]:
BarraFactor_3.to_csv('D:/StockDataBase/BarraFactor_3.csv',index=False)

## 非线性市值因子
$$NLSIZE=ln^3(cap)对ln(cap)中性化处理，横截面回归残差$$

In [209]:
NlsizeDf=DfForStock_V.merge(DerivativeIndicatorDf_0[['S_INFO_WINDCODE','TRADE_DAYS','S_VAL_MV']],on=['S_INFO_WINDCODE','TRADE_DAYS'],how='left')

In [210]:
NlsizeDf['NLSIZE_tmp'] = NlsizeDf['S_VAL_MV'].apply(lambda x: np.log(x)**3)
NlsizeDf['SIZE_tmp'] = NlsizeDf['S_VAL_MV'].apply(lambda x: np.log(x))

In [216]:
def CorssSectionRegNlsize(df):
    df_=df.copy()
    df_1=df.copy().dropna()
    y=df_1['NLSIZE_tmp']
    x=df_1['SIZE_tmp']
    Nlsize=sm.OLS(y,x).fit().resid
    df_1['NLSIZE']=Nlsize
    df_2=df_.merge(df_1,on=['S_INFO_WINDCODE','TRADE_DAYS'],how='left')
    return df_2[['S_INFO_WINDCODE','TRADE_DAYS','NLSIZE']].copy()
NlsizeDf_0=NlsizeDf.groupby('TRADE_DAYS').apply(CorssSectionRegNlsize).reset_index(drop=True)

In [218]:
BarraFactor_4=BarraFactor_3.merge(NlsizeDf_0[['S_INFO_WINDCODE','TRADE_DAYS','NLSIZE']],on=['S_INFO_WINDCODE','TRADE_DAYS'],how='left')

In [223]:
BarraFactor_4.to_csv('D:/StockDataBase/BarraFactor_4.csv',index=False)

## 账面市值比因子
BM=最新一个季度公布的净资产/当前市值

In [226]:
BM_0=DerivativeIndicatorDf_0.copy()
BM_0['BM']=1/BM_0['S_VAL_PB_NEW']

In [229]:
BarraFactor_5=BarraFactor_4.merge(BM_0[['S_INFO_WINDCODE','TRADE_DAYS','BM']],on=['S_INFO_WINDCODE','TRADE_DAYS'],how='left')

## 流动性因子
$$LIQUIDITY=0.35stom+0.35stoq+0.3stoa$$
$$stom_t=ln(\sum_{i=0}^{20}turn_{t-i})$$
$$stoq_t=ln(\frac{1}{3}\sum_{i=0}^{62}turn_{t-i})$$
$$stoa_t=ln(\frac{1}{12}\sum_{i=0}^{251}turn_{t-i})$$

In [233]:
def ComputeLiquidity(df):
    df_=df.sort_values('TRADE_DAYS').copy()
    try:
        FREETURNOVER=df_['S_DQ_FREETURNOVER'][-252:]
        stom=np.log(FREETURNOVER[-21:].sum())
        stoq=np.log(FREETURNOVER[-63:].sum()/3)
        stoa=np.log(FREETURNOVER[-252:].sum()/12)
        LIQUIDITY=0.35*stom+0.35*stoq+0.3*stoa
    except:
        LIQUIDITY=np.nan
    return pd.DataFrame([df_['S_INFO_WINDCODE'].tolist()[0],LIQUIDITY],index=['S_INFO_WINDCODE','LIQUIDITY']).T

In [236]:
LIQUIDITY_0=ComputeMfWithDData(TradeDateDf_M_Used,DerivativeIndicatorDf_0,ComputeLiquidity,StartN=13)
LIQUIDITY_0['LIQUIDITY']=LIQUIDITY_0['LIQUIDITY'].apply(lambda x: np.nan if abs(x)==np.inf else x)

In [241]:
BarraFactor_6=BarraFactor_5.merge(LIQUIDITY_0[['S_INFO_WINDCODE','TRADE_DAYS','LIQUIDITY']],on=['S_INFO_WINDCODE','TRADE_DAYS'],how='left')

In [244]:
BarraFactor_6.to_csv('D:/StockDataBase/BarraFactor_6.csv',index=False)

## 盈利预期因子
$$EARNING=0.68epfwd+0.21cetop+0.11etop$$
$$epfwd 盈利预期市值比$$
$$cetop 现金流量市值比$$
$$etop 盈利市值比$$

In [242]:
EarningEstDf = pd.read_csv('D:/DWDB/%s.csv'%('AShareEarningEst'))

In [247]:
EarningEstDf_0=MdateToMENDdate(EarningEstDf,TradeDateDf_M_Used,'EST_DT')

In [250]:
EarningEstDf_1=EarningEstDf_0.groupby(['S_INFO_WINDCODE','EST_DT']).mean().reset_index()

In [253]:
EarningEstDf_1

,S_INFO_WINDCODE,EST_DT,REPORTING_PERIOD,EST_EPS_DILUTED,EST_NET_PROFIT,EST_MAIN_BUS_INC,EST_EBIT,EST_EBITDA,EST_BASE_CAP,ANN_DT,S_EST_CPS,S_EST_DPS,S_EST_BPS,S_EST_EBT,S_EST_ROA,S_EST_ROE,S_EST_OPROFIT,S_EST_EPSDILUTED,S_EST_EPSBASIC,S_EST_OC,S_EST_NPCAL,S_EST_EPSCAL,S_EST_NPRATE,S_EST_EPSRATE,S_EST_PE,S_EST_PB,S_EST_EVEBITDA,S_EST_DIVIDENDYIELD,S_EST_ENDDATE,S_EST_OPE
0,000001.SZ,2005-01-31,2.005323e+07,0.361400,8.622800e+04,NaN,NaN,NaN,2.022518e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.486067e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.005071e+07,NaN
1,000001.SZ,2005-02-28,2.004990e+07,0.303800,6.144233e+04,NaN,NaN,NaN,1.945868e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.853692e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.005269e+07,NaN
2,000001.SZ,2005-03-31,2.005123e+07,0.333333,NaN,NaN,NaN,NaN,1.945822e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.486067e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.005090e+07,NaN
3,000001.SZ,2005-04-29,2.005456e+07,0.242222,4.500000e+04,1.067800e+06,NaN,NaN,1.941319e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.448300e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.005415e+07,NaN
4,000001.SZ,2005-05-31,2.005578e+07,0.191818,2.326800e+04,8.576345e+05,NaN,NaN,1.970088e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.086233e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.005112e+07,NaN
5,000001.SZ,2005-06-30,2.005552e+07,0.212857,4.117155e+04,9.325515e+05,NaN,NaN,1.939161e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.151100e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.005390e+07,NaN
6,000001.SZ,2005-07-29,2.005373e+07,0.247212,4.869410e+04,NaN,NaN,NaN,1.945663e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.792233e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.006041e+07,NaN
7,000001.SZ,2005-08-31,2.005819e+07,0.166087,3.609700e+04,8.760039e+05,NaN,NaN,1.943236e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.140400e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.006021e+07,NaN
8,000001.SZ,2005-09-30,2.005578e+07,0.191727,4.233225e+04,NaN,NaN,NaN,1.945952e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.444100e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.006053e+07,NaN
9,000001.SZ,2005-10-31,2.005456e+07,0.224556,4.385571e+04,1.024900e+06,NaN,NaN,1.944238e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.290500e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.006056e+07,NaN


In [245]:
EarningEstDf.groupby(['S_INFO_WINDCODE',])

,S_INFO_WINDCODE,WIND_CODE,RESEARCH_INST_NAME,ANALYST_NAME,EST_DT,REPORTING_PERIOD,EST_EPS_DILUTED,EST_NET_PROFIT,EST_MAIN_BUS_INC,EST_EBIT,EST_EBITDA,EST_BASE_CAP,ANN_DT,S_EST_CPS,S_EST_DPS,S_EST_BPS,S_EST_EBT,S_EST_ROA,S_EST_ROE,S_EST_OPROFIT,S_EST_EPSDILUTED,S_EST_EPSBASIC,S_EST_OC,S_EST_NPCAL,S_EST_EPSCAL,S_EST_NPRATE,S_EST_EPSRATE,S_EST_PE,S_EST_PB,S_EST_EVEBITDA,S_EST_DIVIDENDYIELD,S_EST_ENDDATE,S_EST_OPE
0,000792.SZ,000792.SZ,中金公司,时雪松,20050815,20051231,0.7800,59500.0,186900.0,NaN,NaN,7.628205e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20060215.0,NaN
1,000792.SZ,000792.SZ,中金公司,时雪松,20050816,20051231,0.7800,NaN,NaN,NaN,NaN,7.675500e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.986900e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20060216.0,NaN
2,000792.SZ,000792.SZ,中金公司,NaN,20050905,20051231,0.7800,NaN,NaN,NaN,NaN,7.675500e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.986900e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20060305.0,NaN
3,000792.SZ,000792.SZ,中金公司,NaN,20051017,20051231,0.7800,NaN,NaN,NaN,NaN,7.675500e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.986900e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20060417.0,NaN
4,000792.SZ,000792.SZ,中金公司,时雪松,20051024,20051231,0.7400,56800.0,149100.0,NaN,NaN,7.675676e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20060424.0,NaN
5,000792.SZ,000792.SZ,中金公司,时雪松,20051115,20051231,0.7400,NaN,NaN,NaN,NaN,7.675500e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.679900e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20060515.0,NaN
6,000792.SZ,000792.SZ,中金公司,"郦彬,邱劲",20050509,20061231,0.7900,NaN,NaN,NaN,NaN,7.675500e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.063600e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20051109.0,NaN
7,000792.SZ,000792.SZ,中金公司,张金涛,20050516,20061231,0.7900,NaN,NaN,NaN,NaN,7.675500e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.063600e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20051116.0,NaN
8,000792.SZ,000792.SZ,中金公司,NaN,20050523,20061231,0.8000,NaN,NaN,NaN,NaN,7.675500e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.140400e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20051123.0,NaN
9,000792.SZ,000792.SZ,中金公司,张金涛,20050616,20061231,0.8000,NaN,NaN,NaN,NaN,7.675500e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.140400e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20051216.0,NaN


In [243]:
EarningEstDf

,S_INFO_WINDCODE,WIND_CODE,RESEARCH_INST_NAME,ANALYST_NAME,EST_DT,REPORTING_PERIOD,EST_EPS_DILUTED,EST_NET_PROFIT,EST_MAIN_BUS_INC,EST_EBIT,EST_EBITDA,EST_BASE_CAP,ANN_DT,S_EST_CPS,S_EST_DPS,S_EST_BPS,S_EST_EBT,S_EST_ROA,S_EST_ROE,S_EST_OPROFIT,S_EST_EPSDILUTED,S_EST_EPSBASIC,S_EST_OC,S_EST_NPCAL,S_EST_EPSCAL,S_EST_NPRATE,S_EST_EPSRATE,S_EST_PE,S_EST_PB,S_EST_EVEBITDA,S_EST_DIVIDENDYIELD,S_EST_ENDDATE,S_EST_OPE
0,000792.SZ,000792.SZ,中金公司,时雪松,20050815,20051231,0.7800,59500.0,186900.0,NaN,NaN,7.628205e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20060215.0,NaN
1,000792.SZ,000792.SZ,中金公司,时雪松,20050816,20051231,0.7800,NaN,NaN,NaN,NaN,7.675500e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.986900e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20060216.0,NaN
2,000792.SZ,000792.SZ,中金公司,NaN,20050905,20051231,0.7800,NaN,NaN,NaN,NaN,7.675500e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.986900e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20060305.0,NaN
3,000792.SZ,000792.SZ,中金公司,NaN,20051017,20051231,0.7800,NaN,NaN,NaN,NaN,7.675500e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.986900e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20060417.0,NaN
4,000792.SZ,000792.SZ,中金公司,时雪松,20051024,20051231,0.7400,56800.0,149100.0,NaN,NaN,7.675676e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20060424.0,NaN
5,000792.SZ,000792.SZ,中金公司,时雪松,20051115,20051231,0.7400,NaN,NaN,NaN,NaN,7.675500e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.679900e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20060515.0,NaN
6,000792.SZ,000792.SZ,中金公司,"郦彬,邱劲",20050509,20061231,0.7900,NaN,NaN,NaN,NaN,7.675500e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.063600e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20051109.0,NaN
7,000792.SZ,000792.SZ,中金公司,张金涛,20050516,20061231,0.7900,NaN,NaN,NaN,NaN,7.675500e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.063600e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20051116.0,NaN
8,000792.SZ,000792.SZ,中金公司,NaN,20050523,20061231,0.8000,NaN,NaN,NaN,NaN,7.675500e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.140400e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20051123.0,NaN
9,000792.SZ,000792.SZ,中金公司,张金涛,20050616,20061231,0.8000,NaN,NaN,NaN,NaN,7.675500e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.140400e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20051216.0,NaN
